# 範例 : (Kaggle)房價預測
***
- 以下用房價預測資料, 觀察均值編碼的效果

# [教學目標]
- 以下用房價預測資料, 觀察均值編碼的效果

# [範例重點]
- 觀察標籤編碼與均值編碼, 在特徵數量 / 線性迴歸分數 / 線性迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

使用線性回歸時，均值編碼可以在幾乎相同的時間內提高預測精準度。

- 觀察標籤編碼與均值編碼, 在特徵數量 / 梯度提升樹分數 / 梯度提升樹時間上, 分別有什麼影響 (In[5], Out[5], In[6], Out[6]) 

使用均值編碼並不會相對提高太多梯度提升樹的預測精準度，時間也會被拉長

In [4]:
import pandas as pd
import numpy as np 
import copy,time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from  sklearn.model_selection import cross_val_score

data_path = 'C:/Users/francis/Machine_Learning/francis/Documents/GitHub/4th-ML100Days/data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')
df_train.head()
Y_train = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id','SalePrice'] , axis = 1)
df_test = df_test.drop(['Id'] , axis = 1)
df = pd.concat([df_train,df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [5]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features} \n')
# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = Y_train.shape[0]
df.head()

43 Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] 



,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [6]:
# 對照組 : 標籤編碼 + 線性迴歸
df_temp = pd.DataFrame()
for col in df.columns:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
X_train = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator,X_train,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.6615606866851304
time : 0.08527469635009766 sec


In [7]:
# 均值編碼 + 線性迴歸
data = pd.concat([df[:train_num],Y_train] , axis = 1)
data
#先將col內的類別所對應的SalePrice，用groupby統整，再將其對應的值做平均，算出col裡面不同類別的值對應的SalePrice值平均
#做reset_index是為了不讓col的類別變成 index
#使用pd.merge合併資料
#on之所以會定為該column，是為了讓column_mean可以對應到相應的值，利用merge把column當作連接點，將兩個Data_Frame串接起來
#how = left意思是只保留左邊的數據的對應值，ex:只取用data的column值，inner代表左右dataframe的交集，out代表聯集
#最後再用drop將原本的物件column拿掉，column_mean取代column
for col in df.columns:
    mean_df = data.groupby(col)['SalePrice'].mean().reset_index()
    mean_df.columns = [col,f'{col}_mean']
    data = pd.merge(data , mean_df , how = 'left')
    data = data.drop([col] , axis = 1)
data = data.drop('SalePrice',axis = 1)
estimator = LinearRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator,data,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')
''''''''''    
mean_df = data.groupby('Street')['SalePrice'].mean().reset_index()
mean_df.columns = ['Street' , 'Street_mean']
data = pd.merge(data,mean_df,on = 'Street', how =  'left')
data.drop(['Street'], axis = 1)
'''''''''

shape : (1460, 43)
score : 0.7624230403716974
time : 0.04986691474914551 sec


"'    \nmean_df = data.groupby('Street')['SalePrice'].mean().reset_index()\nmean_df.columns = ['Street' , 'Street_mean']\ndata = pd.merge(data,mean_df,on = 'Street', how =  'left')\ndata.drop(['Street'], axis = 1)\n"

In [8]:
# 對照組 : 標籤編碼 + 梯度提升樹
df_temp = pd.DataFrame()
for col in df.columns:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
X_train = df_temp[:train_num]
estimator_1 = GradientBoostingRegressor()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator_1,X_train,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.7773323844950724
time : 1.288538932800293 sec


In [9]:
# 均值編碼 + 梯度提升樹
data = pd.concat([df[:train_num],Y_train],axis = 1)
for col in df.columns:
    mean_df = data.groupby(col)['SalePrice'].mean().reset_index()
    mean_df.columns = [col,f'{col}_mean']
    data = pd.merge(data,mean_df,on = col , how = 'left')
    data = data.drop([col],axis = 1)
data = data.drop(['SalePrice'] ,axis = 1)
estimator_1 = GradientBoostingRegressor()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator_1,data,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start}')

shape : (1460, 43)
score : 0.8066034054503731
time : 0.9289708137512207


# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

用均值編碼的效果，我認為是因為有些物件的特徵類別並沒有順序之分，一旦將其轉為從0到後面的數字，會影響預測的準確度，而均值編碼本身是根據其對應Y值的平均算出，所以值會跟Y值有正向的關係。

In [10]:
import pandas as pd
import numpy as np 
import copy,time
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.head()
Y_train = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId','Survived'], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)
df = pd.concat([df_train,df_test] ,axis = 0)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
object_features_1 = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == 'object':
        object_features_1.append(feature)
print(f'{len(object_features_1)} Object Features : {object_features_1} \n')

df = df[object_features_1]
train_num = Y_train.shape[0]
df = df.fillna('None')
df.columns

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'] 



Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [12]:
# 對照組 : 標籤編碼 + 線性迴歸
df_temp = pd.DataFrame()
for col in df.columns:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
X_train = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator,X_train,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.33494273995838764
time : 0.0608372688293457 sec


In [13]:
# 均值編碼 + 線性迴歸
data = pd.concat([df[:train_num],Y_train] ,axis = 1)
data.head()
for col in df.columns:
    mean_df = data.groupby(col)['Survived'].mean().reset_index()
    mean_df.columns = [col,f'{col}_mean']
    data = pd.merge(data,mean_df,on = col , how = 'left')
    data = data.drop([col], axis = 1)
data.head()
data.drop(['Survived'], axis = 1)
estimator = LinearRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator,data,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 6)
score : 1.0
time : 0.031913042068481445 sec


In [14]:
# 對照組 : 標籤編碼 + 梯度提升樹
df_temp = pd.DataFrame()
for col in df.columns:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
X_train = df_temp[:train_num]
estimator_1 = GradientBoostingRegressor()
start = time.time()
print(f'shape : {X_train.shape}')
print(f'score : {cross_val_score(estimator_1,X_train,Y_train,cv = 5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.37627110418594
time : 0.2403576374053955 sec


In [15]:
# 均值編碼 + 梯度提升樹
data = pd.concat([df[:train_num],Y_train], axis = 1)
for col in df.columns:
    mean_df = data.groupby(col)['Survived'].mean().reset_index()
    mean_df.columns = [col,f'{col}_mean']
    data = pd.merge(data,mean_df,on = col ,how = 'left')
    data = data.drop([col],axis = 1)
data = data.drop(['Survived'], axis = 1)
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator_1,data,Y_train,cv  = 5).mean()}')
print(f'time : {time.time() - start}')

shape : (891, 5)
score : 0.9999996032710972
time : 0.11868071556091309


In [1]:
import time
words = input('Please input the words you want to say!:')
for item in words.split():
    item = item + ' '
    letterlist = []
    for y in range(12, -12, -1):
        list_X = []
        letters = ''
        for x in range(-30, 30):
            expression = ((x*0.05)**2+(y*0.1)**2-1)**3-(x*0.05)**2*(y*0.1)**3
            if expression <= 0:
                letters += item[(x-y) % len(item)]
            else:
                letters += ' '
        list_X.append(letters)
        letterlist += list_X
    print('\n'.join(letterlist))
    time.sleep(1.5);

Please input the words you want to say!:to be with you 100 days I Love You bae
                o to to t           to to to                
            o to to to to to    to to to to to to           
          to to to to to to to to to to to to to to         
         to to to to to to to to to to to to to to t        
        to to to to to to to to to to to to to to to        
        o to to to to to to to to to to to to to to t       
         to to to to to to to to to to to to to to to       
        to to to to to to to to to to to to to to to        
        o to to to to to to to to to to to to to to t       
         to to to to to to to to to to to to to to to       
         o to to to to to to to to to to to to to to        
          to to to to to to to to to to to to to to         
          o to to to to to to to to to to to to to          
            o to to to to to to to to to to to to           
             to to to to to to to to to to to to            
      

                I I I I I           I I I I I               
             I I I I I I I I     I I I I I I I I            
          I I I I I I I I I I I I I I I I I I I I I         
         I I I I I I I I I I I I I I I I I I I I I I        
        I I I I I I I I I I I I I I I I I I I I I I I       
         I I I I I I I I I I I I I I I I I I I I I I        
        I I I I I I I I I I I I I I I I I I I I I I I       
         I I I I I I I I I I I I I I I I I I I I I I        
        I I I I I I I I I I I I I I I I I I I I I I I       
         I I I I I I I I I I I I I I I I I I I I I I        
          I I I I I I I I I I I I I I I I I I I I I         
           I I I I I I I I I I I I I I I I I I I I          
          I I I I I I I I I I I I I I I I I I I I I         
             I I I I I I I I I I I I I I I I I I            
              I I I I I I I I I I I I I I I I I             
               I I I I I I I I I I I I I I I I              
                I I I I 